In [4]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import scipy.sparse
import gzip
import xml.etree.ElementTree as ET
from random import sample 
import math
import subprocess
import random
from collections import defaultdict
import glob, os, sys
import networkx as nx
from sklearn import svm
from sklearn.model_selection import train_test_split
import scipy
import json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
import seaborn as sns

In [5]:
np.random.seed(1)
malware_positions = glob.glob('/datasets/dsc180a-wi20-public/Malware/amd_data_smali/*/*/*')
benign_positions = glob.glob('/datasets/home/home-00/11/911/s3mai/malware/Data/large_app/entertainment/*')
# malware_positions = list(np.random.choice(malware_positions, 500, replace = False))
malware_type = glob.glob('/datasets/dsc180a-wi20-public/Malware/amd_data_smali/*')
malware_type = [re.findall("/([A-Za-z]*)", i)[-1] for i in malware_type]

In [6]:
position = '/datasets/dsc180a-wi20-public/Malware/amd_data_smali/'
num_type_malwares = [len(glob.glob(position + i + "/*/*")) for i in malware_type]

In [7]:
pd.Series(index=malware_type, data=num_type_malwares)

VikingHorde        7
GingerMaster     128
FakeAngry         10
Gorpo             36
Youmi            568
Andup             45
Mtk               67
SimpleLocker     173
Boxer             44
RuMMS            402
DroidKungFu      129
Obad               9
Roop              48
Vidro             23
Ksapp             36
Vmvol             13
Koler             69
Lotoor           329
Steek             12
Fjcon             16
Minimob          203
Nandrobox         76
FakeDoc           21
SpyBubble         10
FakeInst        2172
MobileTX          17
SmsZombie          9
FakeAV             5
Kemoge            15
Kyview           175
Univert           10
BankBot          648
Stealer           25
dtype: int64

In [20]:
def malware_position_by_type(malware_position):
    malware_position
    malware_type = glob.glob(malware_position+'/*')
    malware_type = [re.findall("/([A-Za-z]*)", i)[-1] for i in malware_type]
    position_by_type = [(i, glob.glob(position + i + "/*/*")) for i in malware_type]
    return dict(position_by_type)

In [21]:
#all positions of all malwares with corresponded type
malware_type_position = malware_position_by_type("/datasets/dsc180a-wi20-public/Malware/amd_data_smali")

In [22]:
all_benign_apps_position = glob.glob('/datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/*')

In [23]:
len(all_benign_apps_position)

34

In [24]:
lst_position = []
for i in all_benign_apps_position:
    lst_position += (glob.glob(i+"/*"))

In [40]:
malware_type = 'SimpleLocker'

malware_positions = malware_type_position[malware_type]

np.random.seed(5)

num_benign_apps = len(malware_positions)

benign_positions = list(np.random.choice(lst_position, num_benign_apps, replace = False))

decompiled_apks = benign_positions + malware_positions


np.random.seed(4)

train = np.random.choice(decompiled_apks, int(len(decompiled_apks)*0.8), replace = False)

test = [apk for apk in decompiled_apks if apk not in train]

apk_names_train = [get_name(file) for file in train]
apk_classes_train = [get_class(file) for file in train]

apk_names_test = [get_name(file) for file in test]
apk_classes_test = [get_class(file) for file in test]

print(pd.Series(apk_classes_train).value_counts())
print(pd.Series(apk_classes_test).value_counts())

apk2idx_train = dict(zip(apk_names_train, range(len(apk_names_train))))
apk2idx_test = dict(zip(apk_names_test, range(len(apk_names_test))))
APIs = list(get_all_APIs(train))

API2idx = dict(zip(APIs, range(len(APIs))))
idx2API = dict(zip(range(len(APIs)), APIs))


apk2code_blocks_train, apk2call_train = apk_info_idx(train, API2idx, 'train')

apk2code_blocks_test, apk2call_test = apk_info_idx(test, API2idx, 'test')

matrix_A_train = build_matrix_A(API2idx, apk2call_train, apk2idx_train)

scipy.sparse.save_npz(malware_type+"matrix_A_train.npz", matrix_A_train.tocsr())

matrix_A_test = build_matrix_A(API2idx, apk2call_test, apk2idx_test)

scipy.sparse.save_npz(malware_type+"matrix_A_test.npz", matrix_A_test.tocsr())

matrix_B_train = build_matrix_B(API2idx, apk2code_blocks_train, apk2idx_train)

scipy.sparse.save_npz(malware_type+"matrix_B_train.npz", matrix_B_train.tocsr())

matrix_B_test = build_matrix_B(API2idx, apk2code_blocks_test, apk2idx_test)

scipy.sparse.save_npz(malware_type+"matrix_B_test.npz", matrix_B_test.tocsr())

matrix_P_train = build_matrix_P(idx2API, apk2call_train, apk2idx_train)

scipy.sparse.save_npz(malware_type+"matrix_P_train.npz", matrix_P_train.tocsr())

matrix_P_test = build_matrix_P(idx2API, apk2call_test, apk2idx_test)

scipy.sparse.save_npz(malware_type+"matrix_P_test.npz", matrix_P_test.tocsr())



1    138
0    138
dtype: int64
1    35
0    35
dtype: int64
0.36% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/game_racing/%D8%B4%D8%A7%D8%B1%D8%B9-%D8%A7%D9%84%D9%85%D9%88%D8%AA-death-road
0.72% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/sports/fc-basel-1893
1.09% /datasets/dsc180a-wi20-public/Malware/amd_data_smali/SimpleLocker/variety1/734b5f120ad25e3f1c81365f8339d0f8
1.45% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/health_and_fitness/womanlog-baby-calendar
1.81% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/entertainment/freddy-fazbear-mod
2.17% /datasets/dsc180a-wi20-public/Malware/amd_data_smali/SimpleLocker/variety2/7264eb94edab67141f19438f038f719f
2.54% /datasets/dsc180a-wi20-public/Malware/amd_data_smali/SimpleLocker/variety4/cf856f9f087cbe8fe2051dba2b31eb09
2.90% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/books_and_reference/sabbath-school-audio-quarterly
3.26% /datasets/home/home-0

In [69]:
(matrix_A_train.getnnz()+matrix_A_test.getnnz())/(matrix_A_train.shape[0]+matrix_A_test.shape[0])

10118.988439306358

# AA^T

In [38]:
clf = svm.SVC(kernel='precomputed')
gram = (matrix_A_train * matrix_A_train.T).toarray()
clf.fit(gram, apk_classes_train)
print(clf.score(gram, apk_classes_train))
gram_test = (matrix_A_test * matrix_A_train.T).toarray()
print(clf.score(gram_test, apk_classes_test))


tn, fp, fn, tp = confusion_matrix(apk_classes_test, clf.predict(gram_test)).ravel()
print(tn, fp, fn, tp)
fbeta_score(apk_classes_test, clf.predict(gram_test), beta=0.1069)

1.0
1.0
1 0 0 2


1.0

# ABA^T

In [161]:
clf = svm.SVC(kernel='precomputed')
gram = (matrix_A_train * matrix_B_train * matrix_A_train.T).toarray()
clf.fit(gram, apk_classes_train)
print(clf.score(gram, apk_classes_train))
gram_test = (matrix_A_test * matrix_B_train * matrix_A_train.T).toarray()
print(clf.score(gram_test, apk_classes_test))

tn, fp, fn, tp = confusion_matrix(apk_classes_test, clf.predict(gram_test)).ravel()
print(tn, fp, fn, tp)
fbeta_score(apk_classes_test, clf.predict(gram_test), beta=0.09)

1.0
1.0


# APA^T

In [163]:
clf = svm.SVC(kernel='precomputed')
gram = (matrix_A_train * matrix_P_train * matrix_A_train.T).toarray()
clf.fit(gram, apk_classes_train)
print(clf.score(gram, apk_classes_train))
gram_test = (matrix_A_test * matrix_P_train * matrix_A_train.T).toarray()
print(clf.score(gram_test, apk_classes_test))

tn, fp, fn, tp = confusion_matrix(apk_classes_test, clf.predict(gram_test)).ravel()
print(tn, fp, fn, tp)
fbeta_score(apk_classes_test, clf.predict(gram_test), beta= 0.0858)

1.0
0.6666666666666666


# APB(P^T)(A^T)

In [165]:
clf = svm.SVC(kernel='precomputed')
gram = (matrix_A_train * matrix_P_train * matrix_B_train * matrix_P_train.T * matrix_A_train.T).toarray()
clf.fit(gram, apk_classes_train)
print(clf.score(gram, apk_classes_train))
gram_test = (matrix_A_test * matrix_P_test * matrix_B_train * matrix_P_train.T * matrix_A_train.T).toarray()
print(clf.score(gram_test, apk_classes_test))

tn, fp, fn, tp = confusion_matrix(apk_classes_test, clf.predict(gram_test)).ravel()
print(tn, fp, fn, tp)
fbeta_score(apk_classes_test, clf.predict(gram_test), beta= 0.0565)

1.0
0.6666666666666666


In [11]:
def compute_score_of_all_malwares(lst_position, malwares_types = ["all"], malware_type_position=malware_type_position):
    
    rows = []

    if "all" in malwares_types:
        malware_positions = list(malware_type_position.keys())
        
    for i in malwares_types: 
        
        print(i)
        malware_positions = malware_type_position[i]

        np.random.seed(5)

        num_benign_apps = len(malware_positions)
        
        
        if len(lst_position) <= num_benign_apps:
            
            num_benign_apps = len(lst_position)
            
        elif len(lst_position) > num_benign_apps:
            
            num_benign_apps = len(lst_position)
        
        print(num_benign_apps)
        
        benign_positions = list(np.random.choice(lst_position, num_benign_apps, replace = False))
        
        print(len(benign_positions))

        decompiled_apks = benign_positions + malware_positions

        np.random.seed(4)

        train = np.random.choice(decompiled_apks, int(len(decompiled_apks)*0.8), replace = False)

        test = [apk for apk in decompiled_apks if apk not in train]

        apk_names_train = [get_name(file) for file in train]
        apk_classes_train = [get_class(file) for file in train]

        apk_names_test = [get_name(file) for file in test]
        apk_classes_test = [get_class(file) for file in test]

        print(pd.Series(apk_classes_train).value_counts())
        print(pd.Series(apk_classes_test).value_counts())

        apk2idx_train = dict(zip(apk_names_train, range(len(apk_names_train))))
        apk2idx_test = dict(zip(apk_names_test, range(len(apk_names_test))))
        APIs = list(get_all_APIs(train))
        global API2idx
        global idx2API
        API2idx = dict(zip(APIs, range(len(APIs))))
        idx2API = dict(zip(range(len(APIs)), APIs))
        
        
        lst = [i, len(malware_positions), num_benign_apps]
         
        apk2code_blocks_train, apk2call_train = apk_info_idx(train, API2idx, 'train')

        apk2code_blocks_test, apk2call_test = apk_info_idx(test, API2idx, 'test')

        matrix_A_train = build_matrix_A(API2idx, apk2call_train, apk2idx_train)

        matrix_A_test = build_matrix_A(API2idx, apk2call_test, apk2idx_test)

        matrix_B_train = build_matrix_B(API2idx, apk2code_blocks_train, apk2idx_train)

        matrix_B_test = build_matrix_B(API2idx, apk2code_blocks_test, apk2idx_test)

        matrix_P_train = build_matrix_P(idx2API, apk2call_train, apk2idx_train)

        matrix_P_test = build_matrix_P(idx2API, apk2call_test, apk2idx_test)

        #AA^T
        
        lst1 = lst.copy()
        
        clf = svm.SVC(kernel='precomputed')
        gram = (matrix_A_train * matrix_A_train.T).toarray()
        clf.fit(gram, apk_classes_train)
        
        lst1.append(clf.score(gram, apk_classes_train))
        
        gram_test = (matrix_A_test * matrix_A_train.T).toarray()
        
        lst1.append(clf.score(gram_test, apk_classes_test))
        
        tn, fp, fn, tp = confusion_matrix(apk_classes_test, clf.predict(gram_test)).ravel()
        
        lst1 += [tn, fp, fn, tp]
        lst1.append(fbeta_score(apk_classes_test, clf.predict(gram_test), beta=0.1069))
        lst1.appen('AA^T')
        
        #ABA^T
        lst2 = lst.copy()
        clf = svm.SVC(kernel='precomputed')
        gram = (matrix_A_train * matrix_B_train * matrix_A_train.T).toarray()
        clf.fit(gram, apk_classes_train)
        
        lst2.append(clf.score(gram, apk_classes_train))
        gram_test = (matrix_A_test * matrix_B_train * matrix_A_train.T).toarray()
        lst2.append(clf.score(gram_test, apk_classes_test))

        tn, fp, fn, tp = confusion_matrix(apk_classes_test, clf.predict(gram_test)).ravel()
        
        lst2 += [tn, fp, fn, tp]
        
        lst2.append(fbeta_score(apk_classes_test, clf.predict(gram_test), beta=0.09))
        
        lst1.appen('ABA^T')
        #APA
        lst3 = lst.copy()
        
        clf = svm.SVC(kernel='precomputed')
        gram = (matrix_A_train * matrix_P_train * matrix_A_train.T).toarray()
        clf.fit(gram, apk_classes_train)
        lst3.append(clf.score(gram, apk_classes_train))
        gram_test = (matrix_A_test * matrix_P_train * matrix_A_train.T).toarray()
        lst3.append(clf.score(gram_test, apk_classes_test))

        tn, fp, fn, tp = confusion_matrix(apk_classes_test, clf.predict(gram_test)).ravel()
        lst3 += [tn, fp, fn, tp]
        lst3.append(fbeta_score(apk_classes_test, clf.predict(gram_test), beta=0.0858))
        lst3.appen('APA^T')
        
        #APB(P^T)(A^T)
        lst4 = lst.copy()
        
        clf = svm.SVC(kernel='precomputed')
        gram = (matrix_A_train * matrix_P_train * matrix_B_train * matrix_P_train.T * matrix_A_train.T).toarray()
        clf.fit(gram, apk_classes_train)
        
        lst4.append(clf.score(gram, apk_classes_train))
        gram_test = (matrix_A_test * matrix_P_test * matrix_B_train * matrix_P_train.T * matrix_A_train.T).toarray()
        lst4.append(clf.score(gram_test, apk_classes_test))

        tn, fp, fn, tp = confusion_matrix(apk_classes_test, clf.predict(gram_test)).ravel()
        lst4+=[tn, fp, fn, tp]
        lst4.append(fbeta_score(apk_classes_test, clf.predict(gram_test), beta= 0.0565))
        lst4.append('APB(P^T)(A^T)')

        rows += [lst1, lst2, lst3, lst4]
    return rows

In [16]:
lst_position = []
for i in all_benign_apps_position:
    lst_position += (glob.glob(i+"/*"))

In [17]:
malware_type_position.keys()

dict_keys(['VikingHorde', 'GingerMaster', 'FakeAngry', 'Gorpo', 'Youmi', 'Andup', 'Mtk', 'SimpleLocker', 'Boxer', 'RuMMS', 'DroidKungFu', 'Obad', 'Roop', 'Vidro', 'Ksapp', 'Vmvol', 'Koler', 'Lotoor', 'Steek', 'Fjcon', 'Minimob', 'Nandrobox', 'FakeDoc', 'SpyBubble', 'FakeInst', 'MobileTX', 'SmsZombie', 'FakeAV', 'Kemoge', 'Kyview', 'Univert', 'BankBot', 'Stealer'])

In [ ]:
tables = compute_score_of_all_malwares(lst_position, malwares_types = ['VikingHorde'], malware_type_position=malware_type_position)

VikingHorde
472
472
0    379
1      4
dtype: int64
0    93
1     3
dtype: int64
0.26% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/tools/device-id-for-android
0.52% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/lifestyle/%D9%83%D9%88%D8%B1%D8%B3%D8%A7%D8%AA-%D8%AA%D9%85%D8%A7%D8%B1%D9%8A%D9%86-%D9%83%D9%85%D8%A7%D9%84-%D8%A7%D9%84%D8%A7%D8%AC%D8%B3%D8%A7%D9%85
0.78% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/photography/christian-photo-frame-effects
1.04% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/books_and_reference/comentario-b%C3%ADblico
1.31% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/photography/500px-advancedconfig-for-muzei
1.57% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/shopping/encashit-the-cashback-app
1.83% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic/music_and_audio/drum-pad
2.09% /datasets/home/home-00/11/911/s3mai/malware/Data/large_app_dic

In [30]:
gingerMaster_stats

,0,1,2,3,4,5,6,7,8,9
0,GingerMaster,128,128,1.000000,0.980769,25,0,1,26,0.999566
1,GingerMaster,128,128,0.926471,0.750000,23,2,11,16,0.885332
2,GingerMaster,128,128,1.000000,0.884615,21,4,2,25,0.862504
3,GingerMaster,128,128,1.000000,0.980769,25,0,1,26,0.999878


In [33]:
gingerMaster_stats.columns = ['malware_type', 'num_malwares', 'num_of_benign_apps', 'train_accuracy',
                              'test_accuracy', 'true_negative', 'false_postive', 'false_negative', 
                              'true_positive','F_score']

In [35]:
gingerMaster_stats['kernel'] = ['AA^T', 'ABA^T', 'APA^T', 'APB(P^T)(A^T)']

In [36]:
gingerMaster_stats

,malware_type,num_malwares,num_of_benign_apps,train_accuracy,test_accuracy,true_negative,false_postive,false_negative,true_positive,F_score,kernel
0,GingerMaster,128,128,1.000000,0.980769,25,0,1,26,0.999566,AA^T
1,GingerMaster,128,128,0.926471,0.750000,23,2,11,16,0.885332,ABA^T
2,GingerMaster,128,128,1.000000,0.884615,21,4,2,25,0.862504,APA^T
3,GingerMaster,128,128,1.000000,0.980769,25,0,1,26,0.999878,APB(P^T)(A^T)


In [24]:
def score_on_metapath_AA(matrix_A_train):
    clf = svm.SVC(kernel='precomputed')
    gram = (matrix_A_train * matrix_P_train * matrix_B_train * matrix_P_train.T * matrix_A_train.T).toarray()
    clf.fit(gram, apk_classes_train)

    lst4.append(clf.score(gram, apk_classes_train))
    gram_test = (matrix_A_test * matrix_P_test * matrix_B_train * matrix_P_train.T * matrix_A_train.T).toarray()
    lst4.append(clf.score(gram_test, apk_classes_test))

    tn, fp, fn, tp = confusion_matrix(apk_classes_test, clf.predict(gram_test)).ravel()
    lst4+=[tn, fp, fn, tp]
    lst4.append(fbeta_score(apk_classes_test, clf.predict(gram_test), beta= 0.0565))
    rows += [lst1, lst2, lst3, lst4]

In [16]:
def get_class(file):
    """
    Return 1 if it is malware, 0 if benign
    """
    return int('dsc180a-wi20-public' in file)

def get_name(file):
    """
    Return name of the APP
    """
    return '-'.join(file.split('/')[-3:])

def get_code_blocks(data):
    return re.findall(r'\.method.*?\.end method', data)

def get_API_calls(data):
    return [API_segmentation(line)[1] for line in re.findall(r'(invoke-.*?->.*?)\s', data)]

def API_segmentation(line):
    idx = line.find('}') + 1
    call = line[idx+1:].strip()
    invoke_type = line[:line.find('{')].strip().split('/')[0]
    return (invoke_type, call)

def data_in_app(filepath):
    code_blocks = []
    API_calls = []
    for root, dirs, files in os.walk(filepath):
        if 'smali' in root:
            for fp in [root + '/' + file for file in files if file.endswith('.smali')]:
                with open(fp) as f:
                    data = ' '.join(f.readlines()).replace('\n', '')
                    #code_blocks.extend(get_code_blocks(data))
                    API_calls.extend(get_API_calls(data))
#     return set(code_blocks), set(API_calls)
    return set(API_calls)


# For each decompiled apk, get the smali codes and call2invoke dicitionary
def apk_info(decompiled_apks):
    apk2code_blocks = {}
    apk2call = {}
    counter = 0
    total = len(decompiled_apks)
    for apk in decompiled_apks:
        counter += 1
        print("{:.2f}%".format(counter / total * 100), apk)
        name = get_name(apk)
        code_blocks, API_calls = data_in_app(apk)
        apk2code_blocks[name] = code_blocks
        apk2call[name] = API_calls
    return apk2code_blocks, apk2call

def API_segmentation(line):
    """
    Splits a API line to invoke type and API call
    """
    idx = line.find('}') + 1
    call = line[idx+1:].strip()
    invoke_type = line[:line.find('{')].strip().split('/')[0]
    return (invoke_type, call)


def API_package(API):
    """
    Returns the package of API 
    """
    return API.split('->')[0]


def get_all_APIs(decompiled_apks):
    """
    Returns a set of all APIs in these apks
    """
    APIs = []
    counter = 0
    total = len(decompiled_apks)
    for apk in decompiled_apks:
        counter += 1
        print("{:.2f}%".format(counter / total * 100), apk)
        name = get_name(apk)
#         code_blocks, API_calls = data_in_app(apk)
        API_calls = data_in_app(apk)
        APIs.extend(list(API_calls))
    return set(APIs)

def get_code_blocks_idx(data, API2idx, stage = 'train'):
    """
    Given API2idx mapping, returns a list of lists, where each list is API indices in a same code block
    """
    code_blocks = re.findall(r'\.method.*?\.end method', data)
    API_indices = [get_API_calls_idx(code, API2idx, stage) for code in code_blocks]
    return API_indices

    
def get_API_calls_idx(data, API2idx, stage = 'train'):
    """
    Given API2idx mapping, returns a list API indices in a the code (data)
    If in test stage, ignore APIs not in training data (API2idx)
    """
    if stage == 'train':
        return [API2idx[API_segmentation(line)[1]] for line in re.findall(r'(invoke-.*?->.*?)\s', data)]
    elif stage == 'test':
        result = []
        for line in re.findall(r'(invoke-.*?->.*?)\s', data):
            try:
                result.append(API2idx[API_segmentation(line)[1]])
            except:
                continue
        return result
    else:
        print('Invalid Arguments:', stage)
                
    
def data_in_app_idx(filepath, API2idx, stage = 'train'):
    """
    Given apk filepath and API2idx mapping, returns 
    1) a list of lists, where each list is API indices in a same code block
    2) a set of API indices in the apk
    If in test stage, ignore APIs not in training data (API2idx)
    """
    code_blocks = []
    #API_calls = []
    for root, dirs, files in os.walk(filepath):
        if 'smali' in root:
            for fp in [root + '/' + file for file in files if file.endswith('.smali')]:
                with open(fp) as f:
                    data = ' '.join(f.readlines()).replace('\n', '')
                    code_blocks.extend(get_code_blocks_idx(data, API2idx, stage))
                    #API_calls.extend(get_API_calls_idx(data, API2idx, stage, API_list))
    return code_blocks, set([item for sublist in code_blocks for item in sublist])

def apk_info_idx(decompiled_apks, API2idx, stage = 'train'):
    """
    Given a list of apk filepaths and API2idx mapping, returns 
    1) a dictionary with key = name of apk and value = the first output of data_in_app_idx
    2) a dictionary with key = name of apk and value = the second output of data_in_app_idx
    If in test stage, ignore APIs not in training data (API2idx)
    """
    apk2code_blocks = {}
    apk2call = {}
    counter = 0
    total = len(decompiled_apks)
    for apk in decompiled_apks:
        counter += 1
        print("{:.2f}%".format(counter / total * 100), apk)
        name = get_name(apk)
        code_blocks, API_calls = data_in_app_idx(apk, API2idx, stage)
        apk2code_blocks[name] = code_blocks
        apk2call[name] = API_calls
    return apk2code_blocks, apk2call


def build_matrix_A(API2idx, apk2call, apk2idx):
    """
    Builds matrix A, where A[i,j] is whether APP_i has API_j
    """
#     matrix_A = np.zeros((len(apk2idx), len(API2idx)))
    matrix_A = scipy.sparse.lil_matrix((len(apk2idx), len(API2idx)))
    total = len(apk2idx)
    counter = 0
    for apk in apk2idx:
        counter += 1
        print("{:.2f}%".format(counter / total * 100), apk)
        apk_idx = apk2idx[apk]
        API_indices = apk2call[apk]
        for API_idx in API_indices:
            matrix_A[apk_idx, API_idx] = 1
    return matrix_A

def build_matrix_B(API2idx, apk2code_blocks, apk2idx):
    '''
    Build matrix B, where B[i,j] is whether API_i and API_j occur in the same code block (method).
    '''
    matrix_B = scipy.sparse.lil_matrix((len(API2idx), len(API2idx)))
    total = len(apk2idx)
    counter = 0
    for apk in apk2idx:
        counter += 1
        print("{:.2f}%".format(counter / total * 100), apk)
        code_blocks = apk2code_blocks[apk]
        for block in code_blocks:
            for i in range(len(block)):
                API_i = block[i]
                for j in range(i+1, len(block)):
                    API_j = block[j]
                    matrix_B[API_i, API_j] = 1
                    matrix_B[API_j, API_i] = 1
    return matrix_B

def build_matrix_P(idx2API, apk2call, apk2idx):
    '''
    Build matrix P, where P[i,j] is whether API_i and API_j have the same package.
    '''         
    matrix_P = scipy.sparse.lil_matrix((len(API2idx), len(API2idx)))
    total = len(apk2idx)
    counter = 0
    package2call = defaultdict(set)
    for apk in apk2idx:
        counter += 1
        print("{:.2f}%".format(counter / total * 100), apk)
        for API_idx in apk2call[apk]:
            package2call[API_package(idx2API[API_idx])].add(API_idx)
    total = len(package2call)
    counter = 0
    one_perc = total // 100
    for package in package2call:
        counter += 1
        if counter % one_perc == 0:
            print("{:.2f}%".format(counter / total * 100), package)
        APIs = list(package2call[package])
        for i in range(len(APIs)):
            API_i_idx = APIs[i]
            for j in range(i+1, len(APIs)):
                API_j_idx = APIs[j]
                matrix_P[API_i_idx, API_j_idx] = 1
                matrix_P[API_j_idx, API_i_idx] = 1
    return matrix_P

